In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tf_utils.callbacks import schedule_fn2
from tf_utils.Cifar10DataAdvanced import CIFAR10

In [10]:
IMAGENET_SIZE = 96
IMAGENET_DEPTH = 3
IMAGENET_SHAPE = (IMAGENET_SIZE, IMAGENET_SIZE, IMAGENET_DEPTH)

In [11]:
def build_model(img_shape, num_classes) -> Model:
    base_model = MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=IMAGENET_SHAPE
    )

    num_layers = len(base_model.layers)
    print(f"Number of layers in the base model: {num_layers}")
    fine_tune_at = num_layers - 10
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    input_img = Input(shape=img_shape)
    x = Rescaling(scale=2.0, offset=-1.0)(input_img)
    x = Resizing(height=IMAGENET_SIZE, width=IMAGENET_SIZE)(x)
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs=[input_img],
        outputs=[y_pred]
    )

    model.summary()

    return model

- Best model from chapter 8:   0.7200 accuracy
- Best model from chapter 9_3: 0.8361 accuracy
- Best model from chapter 9_7: 0.8993 accuracy

In [12]:
data = CIFAR10()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

In [13]:
img_shape = data.img_shape
num_classes = data.num_classes

In [14]:
epochs = 100

In [15]:
model = build_model(
    img_shape,
    num_classes
)

opt = Adam(learning_rate=5e-4)

model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

lrs_callback = LearningRateScheduler(
    schedule=schedule_fn2,
    verbose=1
)

es_callback = EarlyStopping(
    monitor="val_loss",
    patience=30,
    verbose=1,
    restore_best_weights=True
)

Number of layers in the base model: 154
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 rescaling_7 (Rescaling)     (None, 32, 32, 3)         0         
                                                                 
 resizing_1 (Resizing)       (None, 96, 96, 3)         0         
                                                                 
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)   

In [16]:
model.fit(
    train_dataset,
    verbose=1,
    epochs=epochs,
    callbacks=[lrs_callback, es_callback],
    validation_data=val_dataset,
)


Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100
262/262 [==============================] - 14s 44ms/step - loss: 0.4832 - accuracy: 0.8414 - val_loss: 1.1506 - val_accuracy: 0.8095 - lr: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/100
262/262 [==============================] - 11s 43ms/step - loss: 0.2628 - accuracy: 0.9086 - val_loss: 1.0192 - val_accuracy: 0.8247 - lr: 0.0010

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/100
262/262 [==============================] - 11s 43ms/step - loss: 0.1798 - accuracy: 0.9368 - val_loss: 0.9359 - val_accuracy: 0.8415 - lr: 0.0010

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/100
262/262 [==============================] - 11s 42ms/step - loss: 0.1209 - accuracy: 0.9590 - val_loss: 0.8723 - val_accuracy: 0.8517 - lr: 0.0010

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/100
262/262 [=======

In [17]:
scores = model.evaluate(
    val_dataset,
    verbose=0
)
print(f"Scores: {scores}")

Scores: [0.5460257530212402, 0.8934545516967773]
